<a href="https://colab.research.google.com/github/Matheus-Pombeiro/transformacao_manipulacao_dados/blob/main/transformacao_manipulacao_dados_desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transformação e manipulação de dados desafio**

## Preaparando o ambiente

In [1]:
# Importando Pandas
import pandas as pd

# Importando NumPy
import numpy as np

## Locação de Imóveis

In [2]:
# Lendo o dataset que contém dados sobre locação de imóveis
locacao_imoveis = pd.read_json('/content/dados_locacao_imoveis.json')
locacao_imoveis.head()

,dados_locacao
0,"{'apartamento': 'A101 (blocoAP)', 'datas_combi..."
1,"{'apartamento': 'A102 (blocoAP)', 'datas_combi..."
2,"{'apartamento': 'B201 (blocoAP)', 'datas_combi..."
3,"{'apartamento': 'B202 (blocoAP)', 'datas_combi..."
4,"{'apartamento': 'C301 (blocoAP)', 'datas_combi..."


In [3]:
# Preparando o dataset
locacao_imoveis = pd.json_normalize(locacao_imoveis['dados_locacao'])
locacao_imoveis.head()

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101 (blocoAP),"[01/06/2022, 01/07/2022]","[05/06/2022, 03/07/2022]","[$ 1000,0 reais, $ 2500,0 reais]"
1,A102 (blocoAP),"[02/06/2022, 02/07/2022]","[02/06/2022, 06/07/2022]","[$ 1100,0 reais, $ 2600,0 reais]"
2,B201 (blocoAP),"[03/06/2022, 03/07/2022]","[07/06/2022, 03/07/2022]","[$ 1200,0 reais, $ 2700,0 reais]"
3,B202 (blocoAP),"[04/06/2022, 04/07/2022]","[07/06/2022, 05/07/2022]","[$ 1300,0 reais, $ 2800,0 reais]"
4,C301 (blocoAP),"[05/06/2022, 05/07/2022]","[10/06/2022, 09/07/2022]","[$ 1400,0 reais, $ 2900,0 reais]"


In [4]:
# Obtendo as colunas com dados em lista
colunas = list(locacao_imoveis.columns)
colunas

['apartamento',
 'datas_combinadas_pagamento',
 'datas_de_pagamento',
 'valor_aluguel']

In [5]:
# Separando os dados
locacao_imoveis = locacao_imoveis.explode(colunas[1:])

# Resetando o index
locacao_imoveis.reset_index(inplace=True, drop=True)
locacao_imoveis.head(3)

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101 (blocoAP),01/06/2022,05/06/2022,"$ 1000,0 reais"
1,A101 (blocoAP),01/07/2022,03/07/2022,"$ 2500,0 reais"
2,A102 (blocoAP),02/06/2022,02/06/2022,"$ 1100,0 reais"


In [6]:
locacao_imoveis.info()

# object == pode ser uma string mesmo contendo números

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   apartamento                 30 non-null     object
 1   datas_combinadas_pagamento  30 non-null     object
 2   datas_de_pagamento          30 non-null     object
 3   valor_aluguel               30 non-null     object
dtypes: object(4)
memory usage: 1.1+ KB


In [7]:
# Removendo caracteres especiais de 'valor_aluguel'
locacao_imoveis['valor_aluguel'] = locacao_imoveis['valor_aluguel'].apply(lambda x: x.replace('$', '').replace('reais', '').replace(',', '').strip())

# Convertendo o tipo de 'valor_aluguel'
locacao_imoveis['valor_aluguel'] = locacao_imoveis['valor_aluguel'].astype(np.float64)
locacao_imoveis.head(3)

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101 (blocoAP),01/06/2022,05/06/2022,10000.0
1,A101 (blocoAP),01/07/2022,03/07/2022,25000.0
2,A102 (blocoAP),02/06/2022,02/06/2022,11000.0


In [8]:
# Substituindo '(blocoAP)' por nada
locacao_imoveis['apartamento'] = locacao_imoveis['apartamento'].str.replace('\(blocoAP\)', '', regex=True)
locacao_imoveis['apartamento'].head(3)

# Foi necessário adicionar a barra '\' para ser considerados os parênteses como caracteres literais

,apartamento
0,A101
1,A101
2,A102


In [19]:
# Convertendo para o tipo datetime no formato dd/mm/yyyy
locacao_imoveis['datas_combinadas_pagamento'] = pd.to_datetime(locacao_imoveis['datas_combinadas_pagamento'], format='%d/%m/%Y')
locacao_imoveis['datas_combinadas_pagamento'].head(3)

,datas_combinadas_pagamento
0,2022-06-01
1,2022-07-01
2,2022-06-02


In [20]:
locacao_imoveis['datas_de_pagamento'] = pd.to_datetime(locacao_imoveis['datas_de_pagamento'], format='%d/%m/%Y')
locacao_imoveis['datas_de_pagamento'].head(3)

,datas_de_pagamento
0,2022-06-05
1,2022-07-03
2,2022-06-02


In [21]:
# Calculando o atraso dos pagamentos
locacao_imoveis['atraso'] = (locacao_imoveis['datas_de_pagamento'] - locacao_imoveis['datas_combinadas_pagamento']).dt.days
locacao_imoveis['atraso'].head(3)

,atraso
0,4
1,2
2,0


In [22]:
# Calculando a média dos atrasos por apartamento
media_atraso = locacao_imoveis.groupby('apartamento')['atraso'].mean()
media_atraso.head(3)

,atraso
apartamento,
A101,3.0
A102,2.0
B201,2.0


## Vendas Online

In [9]:
# Lendo o dataset que contém dados sobre vendas
vendas = pd.read_json('/content/dados_vendas_clientes.json')
vendas.head()

,dados_vendas
0,"{'Data de venda': '06/06/2022', 'Cliente': ['@..."
1,"{'Data de venda': '07/06/2022', 'Cliente': ['I..."
2,"{'Data de venda': '08/06/2022', 'Cliente': ['I..."
3,"{'Data de venda': '09/06/2022', 'Cliente': ['J..."
4,"{'Data de venda': '10/06/2022', 'Cliente': ['M..."


In [10]:
# Preparando o dataset
vendas = pd.json_normalize(vendas['dados_vendas'])
vendas.head()

,Data de venda,Cliente,Valor da compra
0,06/06/2022,"[@ANA _LUCIA 321, DieGO ARMANDIU 210, DieGO AR...","[R$ 836,5, R$ 573,33, R$ 392,8, R$ 512,34]"
1,07/06/2022,"[Isabely JOanes 738, Isabely JOanes 738, Isabe...","[R$ 825,31, R$ 168,07, R$ 339,18, R$ 314,69]"
2,08/06/2022,"[Isabely JOanes 738, JOãO Gabriel 671, Julya m...","[R$ 682,05, R$ 386,34, R$ 622,65, R$ 630,79]"
3,09/06/2022,"[Julya meireles 914, MaRIA Julia 444, MaRIA Ju...","[R$ 390,3, R$ 759,16, R$ 334,47, R$ 678,78]"
4,10/06/2022,"[MaRIA Julia 444, PEDRO PASCO 812, Paulo castr...","[R$ 314,24, R$ 311,15, R$ 899,16, R$ 885,24]"


In [11]:
# Obtendo as colunas
colunas = list(vendas.columns)
colunas

['Data de venda', 'Cliente', 'Valor da compra']

In [12]:
# Separando os dados
vendas = vendas.explode(colunas[1:])

# Resentando o index
vendas.reset_index(inplace=True, drop=True)
vendas.head(3)

,Data de venda,Cliente,Valor da compra
0,06/06/2022,@ANA _LUCIA 321,"R$ 836,5"
1,06/06/2022,DieGO ARMANDIU 210,"R$ 573,33"
2,06/06/2022,DieGO ARMANDIU 210,"R$ 392,8"


In [13]:
vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Data de venda    20 non-null     object
 1   Cliente          20 non-null     object
 2   Valor da compra  20 non-null     object
dtypes: object(3)
memory usage: 608.0+ bytes


In [14]:
# Removendo os caracteres especiais de 'Valor da compra'
vendas['Valor da compra'] = vendas['Valor da compra'].apply(lambda x: x.replace('R$', '')\
                                                            .replace(',', '').strip())

# Convertendo o tipo dos dados
vendas['Valor da compra'] = vendas['Valor da compra'].astype(np.float64)
vendas.head(3)

,Data de venda,Cliente,Valor da compra
0,06/06/2022,@ANA _LUCIA 321,8365.0
1,06/06/2022,DieGO ARMANDIU 210,57333.0
2,06/06/2022,DieGO ARMANDIU 210,3928.0


In [15]:
# Verificando os caracteres presentes na coluna 'Cliente'
vendas['Cliente'].head(3)

,Cliente
0,@ANA _LUCIA 321
1,DieGO ARMANDIU 210
2,DieGO ARMANDIU 210


In [16]:
# Convertendo os caracteres para minúsculo
vendas['Cliente'] = vendas['Cliente'].str.lower()
vendas['Cliente'].head(3)

,Cliente
0,@ana _lucia 321
1,diego armandiu 210
2,diego armandiu 210


In [17]:
# Mantendo apenas letras espaços em branco, substituindo os demais caracteres por espaços em branco
vendas['Cliente'] = vendas['Cliente'].str.replace('[^a-z ]', ' ', regex=True)
vendas['Cliente'].head(3)

# O regex não seleciona todas as letras de a-z e espaços em branco ' '
# Tudo que satisfaz o regex é apagado

,Cliente
0,ana lucia
1,diego armandiu
2,diego armandiu


In [23]:
# Convertendo para o tipo datetime no formato dd/mm/yyyy
vendas['Data de venda'] = pd.to_datetime(vendas['Data de venda'], format='%d/%m/%Y')
vendas['Data de venda'].head(3)

,Data de venda
0,2022-06-06
1,2022-06-06
2,2022-06-06


In [24]:
# Calculando o total das compras
total_compras = vendas.groupby('Cliente')['Valor da compra'].sum()
total_compras.head(3)

,Valor da compra
Cliente,
ana lucia,8365.0
diego armandiu,112495.0
isabely joanes,232930.0
